<a href="https://colab.research.google.com/github/Pantercode/FeminicidiosMG/blob/main/feminicidios_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importando as libs necessarias para o projeto
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format
from pyspark.sql.functions import col, count, when, isnan
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import FloatType, IntegerType, BooleanType, StringType, DateType
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window
import pandas as pd

In [ ]:
# Inicializar o SPark


spark = SparkSession.builder \
  .appName('Violencia_contra_mulher') \
  .config('spark.jars.packages', 'com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.21.1') \
  .getOrCreate()


In [ ]:
#Função que renomear da da colunas
def renamear_coluna(df, old_column_name, new_column_name):
    return df.withColumnRenamed(old_column_name, new_column_name)

In [ ]:
# função que altera a tipagem da coluna
def alterar_tipo_dado(df, coluna, novo_tipo):

    if novo_tipo == 'float':
        df = df.withColumn(coluna, regexp_replace(col(coluna), ',', '.').cast(FloatType()))
    elif novo_tipo == 'int':
        df = df.withColumn(coluna, regexp_replace(col(coluna), ',', '.').cast(FloatType()).cast(IntegerType()))
    elif novo_tipo == 'bool':
        df = df.withColumn(coluna, col(coluna).cast(BooleanType()))
    elif novo_tipo == 'string':
        df = df.withColumn(coluna, col(coluna).cast(StringType()))
    elif novo_tipo == 'date':
        df = df.withColumn(coluna, col(coluna).cast(DateType()))
    else:

        print(f"Tipo de dado '{novo_tipo}' não reconhecido.")
    return df



# Leitura dos datasets

In [ ]:
df_2014 = spark.read.csv('violencia_domestica_2014.csv',sep=';',header=True)

In [ ]:
df_2015 = spark.read.csv('violencia_domestica_2015.csv',sep=';',header=True)

In [ ]:
df_2016 = spark.read.csv('violencia_domestica_2016.csv',sep=';',header=True)

In [ ]:
df_2017 = spark.read.csv('violencia_domestica_2017.csv',sep=';',header=True)

In [ ]:
df_2018 = spark.read.csv('violencia_domestica_2018.csv',sep=';',header=True)

In [ ]:
df_2019 = spark.read.csv('violencia_domestica_2019.csv',sep=';',header=True)

In [ ]:
df_2020 = spark.read.csv('violencia_domestica_2020.csv',sep=';',header=True)

In [ ]:
df_2021 = spark.read.csv('violencia_domestica_2021.csv',sep=';',header=True)

In [ ]:
df_2022 = spark.read.csv('violencia_domestica_2022.csv',sep=';',header=True)

In [ ]:
df_2023 = spark.read.csv('violencia_domestica_2023.csv',sep=';',header=True)

In [ ]:
# Concatenando todos os arquivos
df_violencia_mg = df_2014.union(df_2015) \
                    .union(df_2016).union(df_2017) \
                    .union(df_2018).union(df_2019) \
                    .union(df_2020).union(df_2021) \
                    .union(df_2022).union(df_2023)

In [ ]:
#Contabilizando dados nulos ou branco
for column, dtype in df_violencia_mg.dtypes:
    # Conta valores nulos ou em branco
    null_count = df_violencia_mg.filter(
        (col(column).isNull()) | (col(column) == "") | isnan(column)
    ).count()

    print(f"{column} (tipo: {dtype}) - Nulos/Brancos: {null_count}")

municipio_cod (tipo: string) - Nulos/Brancos: 0
municipio_fato (tipo: string) - Nulos/Brancos: 0
data_fato (tipo: string) - Nulos/Brancos: 0
mes (tipo: string) - Nulos/Brancos: 0
ano (tipo: string) - Nulos/Brancos: 0
risp (tipo: string) - Nulos/Brancos: 0
rmbh (tipo: string) - Nulos/Brancos: 0
natureza_delito (tipo: string) - Nulos/Brancos: 0
tentado_consumado (tipo: string) - Nulos/Brancos: 0
qtde_vitimas (tipo: string) - Nulos/Brancos: 0


In [ ]:
df_violencia_mg = renamear_coluna(df_violencia_mg,"municipio_fato","Cidade")
df_violencia_mg = renamear_coluna(df_violencia_mg,"data_fato","Data_Ocorrencia")
df_violencia_mg = renamear_coluna(df_violencia_mg,"risp","Microrregiao")
df_violencia_mg = renamear_coluna(df_violencia_mg,"rmbh","RMBH")
df_violencia_mg = renamear_coluna(df_violencia_mg,"natureza_delito","Tipo_Fatalidade")
df_violencia_mg = renamear_coluna(df_violencia_mg,"tentado_consumado","Resultado_Fatalidade")
df_violencia_mg = renamear_coluna(df_violencia_mg,"qtde_vitimas","Vitimas")
df_violencia_mg.show()


+-------------+-------------------+---------------+---+----+--------------------+-----------------+--------------------+--------------------+-------+
|municipio_cod|             Cidade|Data_Ocorrencia|mes| ano|        Microrregiao|             RMBH|     Tipo_Fatalidade|Resultado_Fatalidade|Vitimas|
+-------------+-------------------+---------------+---+----+--------------------+-----------------+--------------------+--------------------+-------+
|       310010|ABADIA DOS DOURADOS|     2014-01-01|  1|2014|10º RISP - Patos ...|3) Interior de MG|                DANO|           CONSUMADO|      1|
|       310030|         ABRE-CAMPO|     2014-01-01|  1|2014| 12º RISP - Ipatinga|3) Interior de MG|      LESAO CORPORAL|           CONSUMADO|      1|
|       310040|            ACAIACA|     2014-01-01|  1|2014| 12º RISP - Ipatinga|3) Interior de MG|              AMEACA|           CONSUMADO|      1|
|       310060|           AGUA BOA|     2014-01-01|  1|2014|  14º RISP - Curvelo|3) Interior de MG| 

In [ ]:
#Alterando o tipo das colunas
alterar_tipo_dado(df_violencia_mg,"municipio_cod", "int")
alterar_tipo_dado(df_violencia_mg,"Data_Ocorrencia", "date")
alterar_tipo_dado(df_violencia_mg,"Vitimas", "int")

DataFrame[municipio_cod: string, Cidade: string, Data_Ocorrencia: string, mes: string, ano: string, Microrregiao: string, RMBH: string, Tipo_Fatalidade: string, Resultado_Fatalidade: string, Vitimas: int]

In [ ]:
#selecionado a d_municipio
df_violencia_mg1 = df_violencia_mg.select('municipio_cod','Cidade','Microrregiao','RMBH')

In [ ]:
# Adicionando a coluna de Id_municipio
windowSpec = Window.orderBy(monotonically_increasing_id())
df_violencia_mg1 = df_violencia_mg1.withColumn("id_municipio", row_number().over(windowSpec))
df_violencia_mg1.show()

+-------------+-------------------+--------------------+-----------------+------------+
|municipio_cod|             Cidade|        Microrregiao|             RMBH|id_municipio|
+-------------+-------------------+--------------------+-----------------+------------+
|       310010|ABADIA DOS DOURADOS|10º RISP - Patos ...|3) Interior de MG|           1|
|       310030|         ABRE-CAMPO| 12º RISP - Ipatinga|3) Interior de MG|           2|
|       310040|            ACAIACA| 12º RISP - Ipatinga|3) Interior de MG|           3|
|       310060|           AGUA BOA|  14º RISP - Curvelo|3) Interior de MG|           4|
|       310060|           AGUA BOA|  14º RISP - Curvelo|3) Interior de MG|           5|
|       310150|       ALEM PARAIBA|4º RISP - Juiz de...|3) Interior de MG|           6|
|       310160|            ALFENAS|18º RISP - Poços ...|3) Interior de MG|           7|
|       310160|            ALFENAS|18º RISP - Poços ...|3) Interior de MG|           8|
|       310160|            ALFEN

In [ ]:
#selecionando as colunas da d_Calendario
df_violencia_mg2 = df_violencia_mg.select('Data_Ocorrencia','mes','ano')
df_violencia_mg2.show()

+---------------+---+----+
|Data_Ocorrencia|mes| ano|
+---------------+---+----+
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
|     2014-01-01|  1|2014|
+---------------+---+----+
only showing top 20 rows



In [ ]:
#selecionando as colunas da d_Fatalidade
df_violencia_mg3 = df_violencia_mg.select('Tipo_Fatalidade','Resultado_Fatalidade')

In [ ]:
#adicionadndo Id a tabela d_fatalidade
windowSpec3 = Window.orderBy(monotonically_increasing_id())
df_violencia_mg3 = df_violencia_mg3.withColumn("id_fatalidade", row_number().over(windowSpec))
df_violencia_mg3.show()

+--------------------+--------------------+-------------+
|     Tipo_Fatalidade|Resultado_Fatalidade|id_fatalidade|
+--------------------+--------------------+-------------+
|                DANO|           CONSUMADO|            1|
|      LESAO CORPORAL|           CONSUMADO|            2|
|              AMEACA|           CONSUMADO|            3|
|              AMEACA|           CONSUMADO|            4|
|VIAS DE FATO / AG...|           CONSUMADO|            5|
|VIAS DE FATO / AG...|           CONSUMADO|            6|
|              AMEACA|           CONSUMADO|            7|
|      LESAO CORPORAL|           CONSUMADO|            8|
|VIAS DE FATO / AG...|           CONSUMADO|            9|
|      LESAO CORPORAL|           CONSUMADO|           10|
|           DIFAMACAO|           CONSUMADO|           11|
|VIAS DE FATO / AG...|           CONSUMADO|           12|
|      LESAO CORPORAL|           CONSUMADO|           13|
|      LESAO CORPORAL|           CONSUMADO|           14|
|VIAS DE FATO 

In [ ]:
#selecionando as colunas da Fato
df_violencia_mg4 = df_violencia_mg.select('Vitimas','Data_Ocorrencia')

In [ ]:
#adicionadndo os Id da tabela fato
windowSpec4 = Window.orderBy(monotonically_increasing_id())
df_violencia_mg4 = df_violencia_mg4.withColumn("id_fatalidade", row_number().over(windowSpec))
df_violencia_mg4 = df_violencia_mg4.withColumn("id_municipio", row_number().over(windowSpec))
df_violencia_mg4.show()

+-------+---------------+-------------+------------+
|Vitimas|Data_Ocorrencia|id_fatalidade|id_municipio|
+-------+---------------+-------------+------------+
|      1|     2014-01-01|            1|           1|
|      1|     2014-01-01|            2|           2|
|      1|     2014-01-01|            3|           3|
|      1|     2014-01-01|            4|           4|
|      1|     2014-01-01|            5|           5|
|      3|     2014-01-01|            6|           6|
|      2|     2014-01-01|            7|           7|
|      1|     2014-01-01|            8|           8|
|      3|     2014-01-01|            9|           9|
|      1|     2014-01-01|           10|          10|
|      2|     2014-01-01|           11|          11|
|      1|     2014-01-01|           12|          12|
|      4|     2014-01-01|           13|          13|
|      1|     2014-01-01|           14|          14|
|      2|     2014-01-01|           15|          15|
|      2|     2014-01-01|           16|       

In [ ]:
#Convertendo os arquivos em daframe pandas
d_municipio = df_violencia_mg1.toPandas()
d_Calendario= df_violencia_mg2.toPandas()
d_fatalidade = df_violencia_mg3.toPandas()
Fato_viole_fem = df_violencia_mg4.toPandas()

In [ ]:
#transformando os dataframe e arquivo parquet
d_municipio.to_parquet('d_municipio.parquet')
d_Calendario.to_parquet('d_Calendario.parquet')
d_fatalidade .to_parquet('d_fatalidade.parquet')
Fato_viole_fem.to_parquet('Fato_viole_fem.parquet')

In [ ]:
# Enviando os arquivos para o Google Storage

In [ ]:
#lib de conexão com Google Cloud
#import os
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\User\\Desktop\\ViolenciaDomestica.json"

In [ ]:
"""

from google.cloud import storage

# Crie um cliente do GCS
client = storage.Client()

# O nome do seu "bucket" no GCS
bucket_name = '04121989'

# Lista dos nomes dos arquivos Parquet no sistema de arquivos local
file_names = ['C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_municipio.parquet', 'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_Calendario.parquet', 'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_fatalidade.parquet', 'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\Fato_viole_fem.parquet']

# Obtenha o bucket
bucket = client.get_bucket(bucket_name)

# Itere sobre a lista de arquivos e faça o upload de cada um
for file_name in file_names:
    # O nome do blob no GCS pode ser o mesmo que o nome do arquivo, ou você pode adicionar algum caminho
    blob_name = file_name  # ou 'pasta_no_gcs/' + file_name

    # Crie um novo blob e faça o upload do arquivo
    blob = bucket.blob(blob_name)
    blob.upload_from_filename(file_name)

    print(f"Arquivo {file_name} enviado com sucesso para o bucket {bucket_name} como {blob_name}")
"""

'\nfrom google.cloud import storage\n\n# Crie um cliente do GCS\nclient = storage.Client()\n\n# O nome do seu "bucket" no GCS\nbucket_name = \'04121989\'\n\n# Lista dos nomes dos arquivos Parquet no sistema de arquivos local\nfile_names = [\'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_municipio.parquet\', \'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_Calendario.parquet\', \'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\d_fatalidade.parquet\', \'C:\\Users\\User\\Desktop\\ViolenciaDomestica\\Fato_viole_fem.parquet\']\n\n# Obtenha o bucket\nbucket = client.get_bucket(bucket_name)\n\n# Itere sobre a lista de arquivos e faça o upload de cada um\nfor file_name in file_names:\n    # O nome do blob no GCS pode ser o mesmo que o nome do arquivo, ou você pode adicionar algum caminho\n    blob_name = file_name  # ou \'pasta_no_gcs/\' + file_name\n\n    # Crie um novo blob e faça o upload do arquivo\n    blob = bucket.blob(blob_name)\n    blob.upload_from_filename(file_name)\n\n    print(f